In [ ]:
#pre installment
#!pip install python-dotenv requests

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

#secret variables
CLIENT_ID = os.getenv("CLIENT_ID")
REFRESH_TOKEN = os.getenv("REFRESH_TOKEN")

In [2]:
def refresh_api_token():
    r = requests.post(
        "https://accounts.spotify.com/api/token",
        headers={"Authorization": f'Basic NDczOWRmNWFiOGY0NGI3ZDhhODkxNDgzYjVmNjBlM2I6ZDE3YmY0ZTAyMDQxNDQ3ZWE2NTNjNDcyMjE1MDkyMTU='},
        data={
            "grant_type": "refresh_token",
            "refresh_token": REFRESH_TOKEN,
        }
    )
    #expiration occur
    res = r.json()
    if "error" in res:
        print(res["error_description"])
        return None
    return res["access_token"]

In [3]:
#global variables
global access_token
access_token = refresh_api_token()
url_base = "https://api.spotify.com/v1/"
redirect_uri = "https://github.com/kfleung129/cs4480"
kpop_genres = "0JQ5DAqbMKFGvOw3O4nLAf"
jpop_genres = ""

In [57]:
#read spotify chart dataset
df = pd.read_csv('charts.csv')

In [4]:
def get_albums(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/artists/{id}/albums',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [5]:
def get_tracks(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/albums/{id}/tracks',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [6]:
def get_tracks_play_count(id):
    global access_token
    r = requests.get(
        f'https://api.t4ils.dev/albumPlayCount?albumid={id}',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [7]:
def get_track(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/tracks/{id}/',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [24]:
def get_artist(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/artists/{id}/',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [8]:
def search(name, _type):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/search?q={name}&type={_type}',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [ ]:
kpop_items = search("justin bieber", "artist")["artists"]["items"]
kpop_items[0]

In [ ]:
for i in kpop_items:
    print(f'{i["name"]}: {i["id"]}')

In [ ]:
albums = get_albums(kpop_items[0]['id'])
def get_artist_dataframes():
    for i in albums['items']:
        flag = False
        for k in i['artists']:
            if k['name'] == kpop_items[0]['name']:
                flag = True
                break
        if not flag:
            continue

        album = get_tracks_play_count(i['id'])
        for tracks in album['data']['discs']:
            for track in tracks['tracks']:
                print(f'{track["name"]}: {track.keys()}')
                
get_artist_dataframes()

In [85]:
def get_track_genres(id):
    track = get_track(id)
    track_artists = track['artists']
    res = ''
    for artist in track_artists:
        _id = artist['id']
        genres = get_artist(_id)['genres']
        res = res + ','.join(genres) + ','
    return res[0: len(res) - 1]

In [86]:
hong_kong_100 = df[df['region'] == 'Hong Kong'][0: 100]

In [87]:
for i in hong_kong_100.index:
    track_id = hong_kong_100['url'][i][hong_kong_100['url'][i].rindex('/')+1:]
    print(f'{hong_kong_100["artist"][i]}: {get_track_genres(track_id)}')

The Chainsmokers, Halsey: pop,electropop,etherpop,indie poptimism,pop
The Weeknd, Daft Punk: canadian contemporary r&b,canadian pop,pop,electro,filter house,rock
DJ Snake, Justin Bieber: edm,electronic trap,pop,pop dance,canadian pop,pop
ZAYN, Taylor Swift: pop,uk pop,pop
Maroon 5, Kendrick Lamar: pop,conscious hip hop,hip hop,rap,west coast rap
Bruno Mars: dance pop,pop
Charlie Puth: pop,viral pop,pop,post-teen pop
Machine Gun Kelly: ohio hip hop,pop rap,dance pop,pop
James Arthur: pop,talent show,uk pop
The Weeknd, Daft Punk: canadian contemporary r&b,canadian pop,pop,electro,filter house,rock
The Chainsmokers, Phoebe Ryan: pop,alt z,indie poptimism
Shawn Mendes: canadian pop,pop,viral pop
Clean Bandit: pop,uk dance,uk funky,dance pop,dancehall,pop,pop rap,pop
Alan Walker: electro house
The Chainsmokers: pop,alt z,pop
Dear Jane: cantopop
Major Lazer: dance pop,edm,electro house,moombahton,pop,pop dance,canadian pop,pop,dance pop,danish pop,electropop,pop dance
Dear Jane: cantopop
Cha